<a href="https://colab.research.google.com/github/manavmittal05/Large-Language-Models/blob/main/A2/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Manav Mittal, 2021538
!pip install transformers huggingface_hub
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes
!pip install accelerate
!pip install datasets

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-afq_fy1a
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-afq_fy1a
  Resolved https://github.com/huggingface/transformers.git to commit d8500cd229c9f687c667be3e4556ffdd678aed69
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9734899 sha256=318f965fadf875bb8e6333a44bcb8318c35a087076cec95f658bcd4e8fa8f548
  Stored in directory: /tmp/pip-ephem-wheel-cache-d9cqjo5t/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.4

In [2]:
from huggingface_hub import login
# from google.colab import userdata

# login(userdata.get('HF_TOKEN'))
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")
login(secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import transformers
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [4]:
class Dataset:

  def __init__(self):
    self.dataset = pd.read_parquet("hf://datasets/cais/mmlu/college_mathematics/test-00000-of-00001.parquet")

  def get_zero_shot_prompt(self, idx):
    question = self.dataset.iloc[idx]['question']
    options = self.dataset.iloc[idx]['choices']

    prompt = f"""
    Choose the answer to the given question from below options. No explaination required.

    {question}

    Option 1: {options[0]}
    Option 2: {options[1]}
    Option 3: {options[2]}
    Option 4: {options[3]}

    Answer:"""

    return prompt

  def get_chain_of_thought_prompt(self, idx):
    question = self.dataset.iloc[idx]['question']
    options = self.dataset.iloc[idx]['choices']

    prompt = f"""
    Choose the answer to the given question from below options. Explain the solution step by step.

    {question}

    Option 1: {options[0]}
    Option 2: {options[1]}
    Option 3: {options[2]}
    Option 4: {options[3]}

    Answer:"""

    return prompt


In [5]:
class llm:

  def __init__(self, model_id):
    self.model_id = model_id
    self.pipeline = transformers.pipeline(
        "text-generation",
        model=self.model_id,
        device_map="auto",
        model_kwargs={
          "torch_dtype": torch.bfloat16,
          "quantization_config": {"load_in_4bit": True},
          "low_cpu_mem_usage": True,
        },

    )

  def generate(self, prompt, max_new_tokens):

    messages = [
        {"role": "user", "content": prompt},
    ]

    outputs = self.pipeline(
        messages,
        max_new_tokens=max_new_tokens,
        # do_sample=True,
        # temperature=0.3,
        # top_p=0.9,
    )
    return outputs[0]["generated_text"][-1]["content"]

In [6]:
gemma = llm("google/gemma-2b-it")

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [7]:
phi = llm("microsoft/Phi-3.5-mini-instruct")

config.json:   0%|          | 0.00/3.45k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [8]:
llama = llm("meta-llama/Meta-Llama-3.1-8B-Instruct")

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [9]:
dataset = Dataset()

In [10]:
from tqdm import tqdm

In [11]:
# write code to generate answers for the questions in the dataset and save them in a dataframe columns gemma, phi, llama

df = pd.DataFrame(columns=['gemma', 'phi', 'llama'])

for i in tqdm(range(len(dataset.dataset))):
  zero_shot_prompt = dataset.get_zero_shot_prompt(i)

  gemma_answer = gemma.generate(zero_shot_prompt, 20)
  phi_answer = phi.generate(zero_shot_prompt, 20)
  llama_answer = llama.generate(zero_shot_prompt, 20)

  new_row = {'gemma': gemma_answer, 'phi': phi_answer, 'llama': llama_answer}
  df.loc[len(df)] = new_row

# save the zero-shot answers in a csv file
df.to_csv('zero_shot_answers.csv', index=False)

# write code to generate answers for the questions in the dataset and save them in a dataframe columns gemma, phi, llama

df = pd.DataFrame(columns=['gemma', 'phi', 'llama'])

for i in tqdm(range(len(dataset.dataset))):
  chain_of_thought_prompt = dataset.get_chain_of_thought_prompt(i)

  gemma_answer = gemma.generate(chain_of_thought_prompt, 1024)
  phi_answer = phi.generate(chain_of_thought_prompt, 1024)
  llama_answer = llama.generate(chain_of_thought_prompt, 1024)

  new_row = {'gemma': gemma_answer, 'phi': phi_answer, 'llama': llama_answer}
  df.loc[len(df)] = new_row

# save the chain-of-thought answers in a csv file
df.to_csv('chain_of_thought_answers.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
 10%|█         | 10/100 [16:53

In [12]:
print(dataset.get_zero_shot_prompt(0))


    Choose the answer to the given question from below options. No explaination required.

    Let k be the number of real solutions of the equation e^x + x - 2 = 0 in the interval [0, 1], and let n be the number of real solutions that are not in [0, 1]. Which of the following is true?

    Option 1: k = 0 and n = 1
    Option 2: k = 1 and n = 0
    Option 3: k = n = 1
    Option 4: k > 1

    Answer:


In [13]:
print(dataset.get_chain_of_thought_prompt(0))


    Choose the answer to the given question from below options. Explain the solution step by step.

    Let k be the number of real solutions of the equation e^x + x - 2 = 0 in the interval [0, 1], and let n be the number of real solutions that are not in [0, 1]. Which of the following is true?

    Option 1: k = 0 and n = 1
    Option 2: k = 1 and n = 0
    Option 3: k = n = 1
    Option 4: k > 1

    Answer:


In [14]:
print(llama.generate(dataset.get_zero_shot_prompt(0), 100))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Option 2: k = 1 and n = 0


In [15]:
print(gemma.generate(dataset.get_zero_shot_prompt(0), 100))

Option 3: k = n = 1

The equation has one real solution in the interval [0, 1], and there are no real solutions outside of this interval.


In [16]:
print(phi.generate(dataset.get_zero_shot_prompt(0), 100))

 Option 2: k = 1 and n = 0


In [17]:
print(llama.generate(dataset.get_chain_of_thought_prompt(0), 4096))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


To solve this problem, let's break it down step by step.

1. **Understanding the equation**: The given equation is e^x + x - 2 = 0. This is a transcendental equation, meaning it involves a function that cannot be expressed in terms of elementary functions (like polynomials).

2. **Analyzing the interval [0, 1]**: We are interested in the number of real solutions within the interval [0, 1]. To do this, let's first consider the behavior of the function e^x + x - 2 within this interval.

3. **Behavior of the function**: 
   - At x = 0, e^x + x - 2 = e^0 + 0 - 2 = 1 - 2 = -1. So, the function is negative at x = 0.
   - As x increases from 0, the function e^x + x - 2 also increases, since both e^x and x increase.
   - At x = 1, e^x + x - 2 = e^1 + 1 - 2 = e + 1 - 2 = e - 1. So, the function is positive at x = 1.

4. **Finding the number of real solutions in [0, 1]**: 
   - Since the function is negative at x = 0 and positive at x = 1, it must have at least one real root within the interval 

In [18]:
print(gemma.generate(dataset.get_chain_of_thought_prompt(0), 4096))

**Option 3: k = n = 1**

**Solution:**

To find the number of real solutions, we need to solve the equation for x.

e^x + x - 2 = 0

This is a quadratic equation, which can be solved using the quadratic formula:

x = (-b ± sqrt(b^2 - 4ac)) / 2a

In this case, a = 1, b = 1, and c = -2, so we have:

x = (-1 ± sqrt(1^2 - 4(1)(-2))) / 2(1)

x = (-1 ± sqrt(9)) / 2

Therefore, the solutions are x = 0 and x = 2.

Since x is in the interval [0, 1], only x = 0 is a real solution.

Therefore, **Option 3: k = n = 1** is true.


In [19]:
print(phi.generate(dataset.get_chain_of_thought_prompt(0), 4096))

 To solve this problem, we need to analyze the behavior of the function f(x) = e^x + x - 2 and determine the number of real solutions within the interval [0, 1], as well as the number of real solutions that are not in this interval.

Step 1: Analyze the function's behavior at the endpoints of the interval [0, 1].

f(0) = e^0 + 0 - 2 = 1 - 2 = -1
f(1) = e^1 + 1 - 2 = e - 1

Since e is approximately 2.718, f(1) is positive.

Step 2: Determine the critical points of the function within the interval [0, 1].

To find the critical points, we need to find the derivative of the function and set it equal to zero.

f'(x) = e^x + 1

Setting f'(x) = 0:

e^x + 1 = 0

This equation has no real solutions, as e^x is always positive, and adding 1 to it will always result in a value greater than 1.

Step 3: Analyze the behavior of the function within the interval [0, 1].

Since there are no critical points within the interval, we only need to consider the endpoints. We have:

f(0) = -1
f(1) > 0

The fun